## Binary Classification of Ethical Concerns

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [4]:
%store -r df

In [5]:
dataset = df
dataset.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,app_name,cat1,inex1,clean_content,cat1_clean
0,gp:AOqpTOF5UoM-6ovAjd8ULHKjifvZCeJyoJWi4F_IaPO...,Max Toon,https://play-lh.googleusercontent.com/a-/AOh14...,This is yhr best app ever im littetally addict...,5,0,18.1.3,2020-12-11 19:51:21,NaN,NaN,tiktok,addiction,internal,yhr app im littetally addict thing personally ...,addiction
1,gp:AOqpTOGZ4VkRpsQa_bVYJdVz65yIOLs5jsENbB_aDJe...,Sandy Mason,https://play-lh.googleusercontent.com/-iFGfy5d...,like tik tok because it gives people a chance ...,5,0,17.3.4,2020-08-21 21:32:37,NaN,NaN,tiktok,Other,NaN,tik tok people chance share life story sad hap...,Other
2,gp:AOqpTOF1KZT5ggeQqGpl62-V6QzBxhROn0eutiZMm9l...,izzyiscool,https://play-lh.googleusercontent.com/a-/AOh14...,I love this app like im addicted,5,0,NaN,2020-10-12 1:38:54,NaN,NaN,tiktok,addiction,internal,love app im addict,addiction
3,gp:AOqpTOFRuZB5C5PEpW09xVx3pts_63bcWm9DFf4rajR...,Kristopher Lyons,https://play-lh.googleusercontent.com/a-/AOh14...,"This app allows pedophile acts, underage half ...",1,17,17.9.5,2020-12-03 9:55:01,NaN,NaN,tiktok,inappropriate content,internal,app pedophile act underage half naked girl ina...,inappropriate content
4,gp:AOqpTOFf_z_S6J2LZIb70xQY4oWiR19R0HN4oIVV4MW...,Certified Skillz,https://play-lh.googleusercontent.com/a-/AOh14...,A very good app but I dont like how theres soo...,5,0,NaN,2020-10-13 1:58:36,NaN,NaN,tiktok,accountability,internal,good app dont soo tiktokers commit offense rep...,accountability


In [6]:
df_concern = dataset.query("cat1 not in ['Other', 'none', 'Noise']")
print("number of all reviews:", len(dataset))
print("number of reviews with concerns:", len(df_concern))

number of all reviews: 3101
number of reviews with concerns: 1920


In [7]:
dataset["target"] = np.where(dataset["cat1"].isin(['Other', 'none', 'Noise']), 0, 1)
X = dataset['content']
y = dataset['target']

In [8]:
def factorize_concern(dataset):
    dataset['cat1_id'] = dataset['cat1'].factorize()[0]
    concern_id_df = dataset[['cat1', 'cat1_id']].drop_duplicates().sort_values('cat1_id')
    concern_to_id = dict(concern_id_df.values)
    id_to_concern = dict(concern_id_df[['cat1_id', 'cat1']].values)
    return dataset, concern_id_df, concern_to_id, id_to_concern

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, dataset.index, test_size=0.75, random_state=42)


In [10]:
len(y_test)

2326

In [11]:
dataset

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,app_name,cat1,inex1,clean_content,cat1_clean,target
0,gp:AOqpTOF5UoM-6ovAjd8ULHKjifvZCeJyoJWi4F_IaPO...,Max Toon,https://play-lh.googleusercontent.com/a-/AOh14...,This is yhr best app ever im littetally addict...,5,0,18.1.3,2020-12-11 19:51:21,NaN,NaN,tiktok,addiction,internal,yhr app im littetally addict thing personally ...,addiction,1
1,gp:AOqpTOGZ4VkRpsQa_bVYJdVz65yIOLs5jsENbB_aDJe...,Sandy Mason,https://play-lh.googleusercontent.com/-iFGfy5d...,like tik tok because it gives people a chance ...,5,0,17.3.4,2020-08-21 21:32:37,NaN,NaN,tiktok,Other,NaN,tik tok people chance share life story sad hap...,Other,0
2,gp:AOqpTOF1KZT5ggeQqGpl62-V6QzBxhROn0eutiZMm9l...,izzyiscool,https://play-lh.googleusercontent.com/a-/AOh14...,I love this app like im addicted,5,0,NaN,2020-10-12 1:38:54,NaN,NaN,tiktok,addiction,internal,love app im addict,addiction,1
3,gp:AOqpTOFRuZB5C5PEpW09xVx3pts_63bcWm9DFf4rajR...,Kristopher Lyons,https://play-lh.googleusercontent.com/a-/AOh14...,"This app allows pedophile acts, underage half ...",1,17,17.9.5,2020-12-03 9:55:01,NaN,NaN,tiktok,inappropriate content,internal,app pedophile act underage half naked girl ina...,inappropriate content,1
4,gp:AOqpTOFf_z_S6J2LZIb70xQY4oWiR19R0HN4oIVV4MW...,Certified Skillz,https://play-lh.googleusercontent.com/a-/AOh14...,A very good app but I dont like how theres soo...,5,0,NaN,2020-10-13 1:58:36,NaN,NaN,tiktok,accountability,internal,good app dont soo tiktokers commit offense rep...,accountability,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,gp:AOqpTOEq_J1k1i2jHYXRaGYBfY4NX7PGO63MvChhgR-...,Federation of Gaia,https://play-lh.googleusercontent.com/a-/AOh14...,Great app if you don't mind getting ripped off...,1,0,NaN,2020-04-24 16:03:08,"Hey there, we're always looking to improve you...",2020-04-25 5:42:34,uber,sustainability,external,great app mind rip drive global warm carbon em...,sustainability,1
3097,gp:AOqpTOHLG5pnAGLrH3uwIAsyxS1akNQo-lql0kHPAPL...,Chan dar,https://play-lh.googleusercontent.com/a-/AOh14...,Myself and my colleagues wanted to car pool an...,1,20,3.65.1,2015-08-29 16:09:05,NaN,NaN,uber,sustainability,external,colleague want car pool respective place long ...,sustainability,1
3098,gp:AOqpTOFW3kL7vyrV7EpBLJeH9nV_Wz7SmVU5FRaa2_j...,Sarthak Sharma,https://play-lh.googleusercontent.com/a-/AOh14...,Its 2020 and dark mode is not even rolled out ...,1,2,287.0.0.50.119,2020-09-11 5:36:06,NaN,NaN,facebook,sustainability,external,dark mode roll imagine waste power contribute ...,sustainability,1
3099,gp:AOqpTOGtKpptH8IZsiA_0GV3V5hXN8O4QucEWHZ5988...,eko purnomowidi,https://play-lh.googleusercontent.com/a-/AOh14...,Instagram give a room to share my experiences ...,5,0,166.1.0.42.245,2020-11-12 3:48:05,NaN,NaN,instagram,none,external,instagram give room share experience coffee pe...,none,0


In [12]:
# Beginning with Default Vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
# Setting up fidf X data
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [13]:
models = {}

# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['Random Forest'] = RandomForestClassifier()

# Support Vector Machines
from sklearn.svm import LinearSVC
models['Support Vector Machines'] = LinearSVC()

# Naive Bayes
from sklearn.naive_bayes import MultinomialNB
models['Naive Bayes'] = MultinomialNB(alpha=0.4, fit_prior=False)

# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['Logistic Regression'] = LogisticRegression()

# MLP Classifier
from sklearn.neural_network import MLPClassifier
models['MLP']  = MLPClassifier(
                        hidden_layer_sizes=(15,),
                        random_state=5,
#                         max_iter=3000,
                        alpha=0.0008
                       )

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy, f1score, precision, recall = {}, {}, {}, {}

for key in models.keys():
    
    # Fit the classifier
    models[key].fit(X_train_tfidf, y_train)
    
    # Make predictions
    predictions = models[key].predict(X_test_tfidf)
    
    # Calculate metrics
    accuracy[key] = accuracy_score(predictions, y_test)
    precision[key] = precision_score(predictions, y_test)
    recall[key] = recall_score(predictions, y_test)
    f1score[key] = f1_score(predictions, y_test)

In [15]:
# With all metrics stored, we can use pandas to view the data as a table:

df_model = pd.DataFrame(index=models.keys(), columns=['Accuracy', 'Precision', 'Recall', 'F1-score'])
df_model['Accuracy'] = accuracy.values()
df_model['Precision'] = precision.values()
df_model['Recall'] = recall.values()
df_model['F1-score'] = f1score.values()
df_model.to_csv('binary_classification_results.csv')
df_model

,Accuracy,Precision,Recall,F1-score
Random Forest,0.809974,0.903005,0.815043,0.856773
Support Vector Machines,0.852537,0.896175,0.872921,0.884395
Naive Bayes,0.864144,0.923497,0.868895,0.895364
Logistic Regression,0.828891,0.946721,0.812427,0.874448
MLP,0.854256,0.887295,0.881874,0.884576


In [16]:
rf = models['Naive Bayes']
preds = rf.predict(X_test_tfidf)
preds

array([1, 1, 0, ..., 1, 1, 0])

In [17]:
def add_predict(df, preds, indices_test):
    for ind, pred in zip(indices_test, preds):
        df.at[ind, "predicted_bc"] = int(pred)
    return df

In [18]:
dataset_pred_bc = add_predict(dataset, preds, indices_test)

In [19]:
dataset_pred_bc = dataset_pred_bc[pd.notnull(dataset_pred_bc['predicted_bc'])]
dataset_pred_bc['predicted_bc'].unique()

array([1., 0.])

In [20]:
dataset_pred_bc = dataset_pred_bc[dataset_pred_bc['predicted_bc']==1]
dataset_pred_bc['cat1'].unique()

array(['addiction', 'inappropriate content', 'Other', 'privacy',
       'discrimination', 'spreading false information', 'censorship',
       'safety', 'identity theft', 'cyberbullying/toxicity', 'scam',
       'harmful advertising', 'Noise', 'accountability', 'transparency',
       'accessibility', 'Content theft', 'sustainability', 'none'],
      dtype=object)

In [21]:
%store dataset_pred_bc

Stored 'dataset_pred_bc' (DataFrame)


In [22]:
df_count = dataset_pred_bc.groupby(['cat1', 'target']).clean_content.count().reset_index(name='counts')
df_count

,cat1,target,counts
0,Content theft,1,6
1,Noise,0,11
2,Other,0,181
3,accessibility,1,20
4,accountability,1,103
5,addiction,1,79
6,censorship,1,132
7,cyberbullying/toxicity,1,103
8,discrimination,1,209
9,harmful advertising,1,32


In [23]:
print(len(dataset_pred_bc[dataset_pred_bc['predicted_bc']==1]))
print(len(X_test_tfidf))

1556
2326
